<a href="https://colab.research.google.com/github/sugarforever/Advanced-RAG/blob/main/03_llama_index_multi_doc_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Llama-Index Multi-Document Agent

It's straightforward to build a RAG application on a single document. But when it comes to multiple documents, it's a different story.

In this tutorial, we will learn how to use Llama-Index to build a RAG application that can answer user questions that require cross query on multiple documents.

The core LlamaIndex components required in this tutorial are as follows:

- VectorStoreIndex
- SummaryIndex
- ObjectIndex
- QueryEngineTool
- OpenAIAgent
- FnRetrieverOpenAIAgent


In [1]:
! pip install llama-index -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 900.5/900.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "your valid openai api key"
openai.api_key = os.environ["OPENAI_API_KEY"]

## Example Code

In [3]:
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleDirectoryReader,
    ServiceContext,
)
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.llms import OpenAI

In [4]:
wiki_titles = [
    "Serie A",
    "Premier League",
    "Bundesliga",
    "La Liga",
    "Ligue 1"
]

In [5]:
from pathlib import Path

import requests

for title in wiki_titles:
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "titles": title,
            "prop": "extracts",
            "explaintext": True,
        },
    ).json()
    page = next(iter(response["query"]["pages"].values()))
    wiki_text = page["extract"]

    data_path = Path("data")
    if not data_path.exists():
        Path.mkdir(data_path)

    with open(data_path / f"{title}.txt", "w") as fp:
        fp.write(wiki_text)

In [6]:
leagues_docs = {}
for wiki_title in wiki_titles:
    leagues_docs[wiki_title] = SimpleDirectoryReader(
        input_files=[f"data/{wiki_title}.txt"]
    ).load_data()

In [7]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
service_context = ServiceContext.from_defaults(llm=llm)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from llama_index.agent import OpenAIAgent
from llama_index import load_index_from_storage, StorageContext
from llama_index.node_parser import SentenceSplitter
import os

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

for idx, wiki_title in enumerate(wiki_titles):
    nodes = node_parser.get_nodes_from_documents(leagues_docs[wiki_title])
    all_nodes.extend(nodes)

    if not os.path.exists(f"./data/{wiki_title}"):
        # build vector index
        vector_index = VectorStoreIndex(nodes, service_context=service_context)
        vector_index.storage_context.persist(
            persist_dir=f"./data/{wiki_title}"
        )
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{wiki_title}"),
            service_context=service_context,
        )

    # build summary index
    summary_index = SummaryIndex(nodes, service_context=service_context)
    # define query engines
    vector_query_engine = vector_index.as_query_engine()
    summary_query_engine = summary_index.as_query_engine()

    # define tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    "Useful for questions related to specific aspects of"
                    f" {wiki_title} (e.g. the history, teams "
                    "and performance in EU, or more)."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Useful for any requests that require a holistic summary"
                    f" of EVERYTHING about {wiki_title}. For questions about"
                    " more specific sections, please use the vector_tool."
                ),
            ),
        ),
    ]

    # build agent
    function_llm = OpenAI(model="gpt-4")
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {wiki_title}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    agents[wiki_title] = agent
    query_engines[wiki_title] = vector_index.as_query_engine(
        similarity_top_k=2
    )

In [12]:
# define tool for each document agent
all_tools = []
for wiki_title in wiki_titles:
    wiki_summary = (
        f"This content contains Wikipedia articles about {wiki_title}. Use"
        f" this tool if you want to answer any questions about {wiki_title}.\n"
    )
    doc_tool = QueryEngineTool(
        query_engine=agents[wiki_title],
        metadata=ToolMetadata(
            name=f"tool_{wiki_title.replace(' ', '_')}",
            description=wiki_summary,
        ),
    )
    all_tools.append(doc_tool)

In [10]:
# define an "object" index and retriever over these tools
from llama_index import VectorStoreIndex
from llama_index.objects import ObjectIndex, SimpleToolNodeMapping

tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
obj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex,
)

In [11]:
from llama_index.agent import FnRetrieverOpenAIAgent

top_agent = FnRetrieverOpenAIAgent.from_retriever(
    obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about the European top football leagues.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)

In [13]:
response = top_agent.query("Tell me about the history and UCL performance of La Liga")

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_La_Liga with args: {
  "input": "history"
}
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "history"
}
Got output: During the 1930s, Athletic Club was the dominant team in La Liga, winning several titles. However, in the 1940s, Atlético Madrid, Barcelona, and Valencia emerged as strong clubs. Atlético Madrid won two titles during this decade, while Barcelona and Valencia each won multiple titles. In the 1950s, FC Barcelona continued their success, winning back-to-back La Liga titles and several other trophies. Real Madrid also emerged as a dominant force during this decade, winning three La Liga titles and dominating the newly created European Cup. Real Madrid's dominance continued into the 1960s and 1970s, winning 14 La Liga titles during this period. However, Atlético Madrid and other clubs like Valencia and Barcelona also had successful s

In [14]:
print(response)

La Liga has a rich history dating back to the 1930s. During the early years, Athletic Club was the dominant team, but in the 1940s, Atlético Madrid, Barcelona, and Valencia emerged as strong contenders. The 1950s saw the rise of FC Barcelona and the dominance of Real Madrid, which continued into the 1960s and 1970s.

In terms of UEFA Champions League (UCL) performance, La Liga teams have had a significant impact. Real Madrid, Barcelona, and Atlético Madrid have been particularly successful, with multiple UCL titles to their names. Other Spanish clubs like Sevilla and Valencia have also won international trophies. La Liga consistently sends a significant number of teams to the Champions League group stage, showcasing the league's overall strength and competitiveness.


In [15]:
response = top_agent.query(
    "Please compare Premier League and La Liga in terms of history and UCL performance"
)

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: tool_Premier_League with args: {
"input": "history"
}
STARTING TURN 1
---------------

=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "history"
}
Got output: The Premier League has a rich history since its establishment in 1992 as the FA Premier League. It operates independently from the English Football League and follows a promotion and relegation system with the EFL. The league's inaugural season took place in 1992-1993 with the participation of 22 clubs, and Manchester United emerged as the first champions. Throughout the years, clubs like Manchester United, Arsenal, Chelsea, Liverpool, and Manchester City have had a significant impact on individual matches. In the 2000s, the "Big Four" clubs emerged, which later expanded to the "Big Six" with the inclusion of Tottenham Hotspur and Manchester City. A notable moment in the league's history was Leicester City's remarkable titl

In [16]:
print(response)

In terms of history, the Premier League was established in 1992 and has since become one of the most popular and financially lucrative football leagues in the world. It is known for its competitive nature and has been dominated by clubs like Manchester United, Arsenal, Chelsea, Liverpool, and Manchester City. The league has also seen remarkable moments, such as Leicester City's title victory in the 2015-2016 season.

On the other hand, La Liga has a longer history, dating back to 1929. It has been home to some of the most successful and iconic clubs in football, including Real Madrid and Barcelona. The league has witnessed periods of dominance from various clubs and has seen intense rivalries between teams. Real Madrid has been particularly successful in the Champions League, winning numerous titles.

In terms of UEFA Champions League (UCL) performance, both leagues have had strong showings. English clubs have won a total of 15 UCL titles, making England the second-most successful coun